
# 🧠 From Predicting to Preventing Customer Churn with Gen AI

---

Retailers face increasing customer churn, rising acquisition costs, and fierce competition, making customer retention more critical than ever.
Despite investments in loyalty programs and CRM systems, many struggle with fragmented customer data, ineffective engagement strategies, and the inability to act in real-time—resulting in missed opportunities and declining customer lifetime value.

<div style="background: #f7f7f7; border-left: 5px solid #ff5f46; padding: 20px; margin: 20px 0; font-size: 17px;">
"A recent review of the State of Customer Churn revealed that the <b>average churn rate</b> for the entire retail sector is <b>6.58%</b>, far higher than the <b>ideal rate of 3-5%</b>."[^3]
</div>

<div style="background: #f7f7f7; border-left: 5px solid green; padding: 20px; margin: 20px 0; font-size: 17px;">
"With <b>prescriptive analytics</b> in place, decision makers can quickly extract <b>actionable knowledge</b> from the available data and define <b>effective actions</b> for the future."[^2]
</div>

---

<div style="display: flex; align-items: center; gap: 15px;">
  <img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/liza.png" 
       style="width: 100px; border-radius: 10px;" />
  <div>
    <strong>Liza, a Generative AI Engineer,</strong> is leveraging Databricks AI to build a prescriptive system aimed at reducing customer churn.  
    By harnessing the full power of her company's data platform, she will create tailored advertising for at-risk customers, encouraging new purchases.
  </div>
</div>


### Liza's Goal  

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/cross_demo_assets/Lakehouse_Demo_Team_architecture_5.png?raw=true" style="float: right" width="500px">

Liza wants to develop a GenAI agent that minimizes the time Account Managers spend crafting customized outreach for customers at risk of churning. More specifically, Liza wants to make an agent that can do multiple things, including:

1. Determining if a customer is likely to churn.
2. If that customer is likely to churn, looking up their recent order information.
3. Based on that recent order information, generating custom, tailored marketing messaging in order to re-engage that customer and encourage their future purchases.

Fortunately, on Databricks, building a multi-faceted AI System with tools that chain off of one another is simple. In the below sections, see how we make it happen!

[^1]: https://www.qualtrics.com/experience-management/customer/customer-churn/

[^2]: https://voziq.ai/featured/prescriptive-analytics-for-churn-reduction/

[^3]: https://www.custify.com/blog/customer-churn-guide/

[^4]: https://www.infobip.com/blog/how-generative-ai-can-help-reduce-churn

[^5]: https://www.salesforce.com/sales/analytics/customer-churn/

[^6]: https://graphite-note.com/customer-churn-prevention-predictive-analytics/

[^7]: https://churnzero.com/blog/customer-success-statistics/

[^8]: https://www.b2brocket.ai/blog-posts/ai-powered-churn-prediction


<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=7405609900705693&notebook=%2F05-Generative-AI%2F05.1-Agent-Functions-Creation&demo_name=lakehouse-retail-c360&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-retail-c360%2F05-Generative-AI%2F05.1-Agent-Functions-Creation&version=1">

In [0]:
%run ../_resources/00-setup $reset_all_data=false

## 🔍 Function 1: Customer Churn Prediction  

<div style="display: flex; align-items: center; gap: 15px;">
  <img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/liza.png" 
       style="width: 100px; border-radius: 10px;" />
  <div>
    To build a Generative AI system that reduces customer churn, Liza first needs a function to predict whether a given customer is at risk of churning.  
    Currently, her company's Account Managers manually look up customer IDs to review recent order history.
  </div>
</div>

### 🛠️ Solution  
Liza will develop a function that:
- Accepts a **customer ID** as input.  
- Returns a **predicted churn status**.  

By leveraging Databricks' `ai_query` feature, she will call the **Churn Prediction model** defined in **Section 4** of this demo.


This function utilizes the churn prediction model we defined in [Section 4]($../04-Data-Science-ML/04.1-automl-churn-prediction) of this demo.

In [0]:
%sql
CREATE OR REPLACE FUNCTION churn_predictor(id STRING)
RETURNS STRING
LANGUAGE SQL
COMMENT 'This function determines whether a customer is at risk of churning.'
RETURN
(
    SELECT CASE 
        WHEN ai_query(
            endpoint => 'dbdemos_customer_churn_endpoint', 
            request => named_struct(
                'user_id', user_id,
                'canal', canal,
                'country', country,
                'gender', gender,
                'age_group', age_group,
                'order_count', order_count,
                'total_amount', total_amount,
                'total_item', total_item,
                'last_transaction', last_transaction,
                'platform', platform,
                'event_count', event_count,
                'session_count', session_count,
                'days_since_creation', days_since_creation,
                'days_since_last_activity', days_since_last_activity,
                'days_last_event', days_last_event
            ),
            returnType => 'STRING'
            ) = '0' THEN 'NOT AT RISK'
        ELSE 'AT RISK'
    END
    FROM churn_user_features
    WHERE user_id = id
    LIMIT 1
)

### Example:

Let's ensure that our new tool works. Here we look up a customer we know is at risk of churn:

In [0]:
%sql
SELECT churn_predictor(
    '2d17d7cd-38ae-440d-8485-34ce4f8f3b46'
) AS prediction

## 📊 Function 2: Customer Order Information Lookup  

<div style="display: flex; align-items: center; gap: 15px;">
  <img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/liza.png" 
       style="width: 100px; border-radius: 10px;" />
  <div>
    Now that Liza's first function can identify customers at risk of churning, she needs an additional tool to retrieve their recent purchase behavior.  
    This function will also collect customer demographic data, ensuring that the AI-generated messaging is personalized and more effective at reducing churn.
  </div>
</div>

### 🛠️ Next Step  
In the following section, we define a function that retrieves a customer's recent orders and demographic information.

In [0]:
%sql
CREATE OR REPLACE FUNCTION customer_order_lookup (
  input_user_id STRING
  COMMENT 'User ID of the customer to be searched'
) 
RETURNS TABLE(
  firstname STRING,
  channel STRING,
  country STRING,
  gender INT,
  order_amount INT,
  order_item_count INT,
  last_order_date STRING,
  current_date STRING,
  days_elapsed_since_last_order INT
)
LANGUAGE SQL
COMMENT "This function returns the customer details for a given customer User ID, along with their last order details. The return fields include First Name, Channel (e.g. Mobile App, Phone, or Web App/Browser ), Country of Residence, Gender, Order Amount, Item Count, and the Last Order Date (in format of yyyy-MM-dd). Use this function when a User ID is given." 
RETURN (
  SELECT
    firstname,
    canal as channel,
    country,
    gender,
    o.order_amount,
    o.order_item_count,
    o.last_order_date,
    TO_CHAR(CURRENT_DATE(), 'yyyy-MM-dd') AS current_date,
    DATEDIFF(CURRENT_DATE(), o.last_order_date) AS days_elapsed_since_last_order
  FROM 
    churn_users u
  LEFT JOIN (
    SELECT
      user_id,
      amount AS order_amount,
      item_count AS order_item_count,
      TO_CHAR(creation_date, 'yyyy-MM-dd') AS last_order_date
    FROM churn_orders
    WHERE user_id = input_user_id
    ORDER BY creation_date DESC
    LIMIT 1
  ) o ON u.user_id = o.user_id
  WHERE
    u.user_id = input_user_id
)

### Example:
To verify the function retrieves orders accurately, we perform the following check:

In [0]:
%sql
SELECT * FROM customer_order_lookup('2d17d7cd-38ae-440d-8485-34ce4f8f3b46')

## ✉️ Function 3: Personalized Customer Outreach with Automated Marketing Copy Generation  

<div style="display: flex; align-items: center; gap: 15px;">
  <img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/liza.png" 
       style="width: 100px; border-radius: 10px;" />
  <div>
    Liza now has the ability to identify at-risk customers and retrieve their order history and demographic details.  
    The next step is to generate personalized marketing copy to proactively re-engage these customers.
  </div>
</div>

### 🛠️ Solution  
This function will utilize **LLMs** to create targeted outreach messages, enabling Account Managers to scale their efforts efficiently. Previously, Account Managers spent significant time on personalized customer outreach to reduce churn. Now, with GenAI, their efforts will be far more efficient.

In the next section, we define the function that automates **customer-specific messaging** to reduce churn. This function is the most complex of the three, so we are going to break it down into three distinct steps below.

### Step 1: Define the model prompt

This function will utilize Llama by-default for creating customized marketing copy on the fly. 

Feel free to manipulate and change the below prompt in order to generate marketing copy that you believe would be the most impactful.

In [0]:
# Feel free to change this prompt! Just know that by changing it, your generated copy may be different. Databricks
#. does not guarantee quality or content of the generated copy in this demo.
prompt = """
    Your only role is to write copy for a contemporary apparel retailer. All your responses are a piece of copy, nothing further.\n
    You have access to the following information from the customer and their last order:\n
    - firstname: the first name of the customer
    - channel: the channel of which the customer is using. This could be WEBAPP for Web app or MOBILE for Mobile app
    - country: the customer's country of residence. This field uses string country code, for example FR for France
    - gender: the customer's gender. This field uses 0 for Male and 1 for Female
    - order_amount: the total amount in US dollars of the last order
    - order_item_count: the total item of the last order
    - last_order_date: the date of the last order in format of yyyy-MM-dd
    - current_date: the current date in format of yyyy-MM-dd
    - days_elapsed_since_last_order: the number of days elapsed since the last order
    - churn: either the customer is AT RISK or NOT AT RISK at churning

    These are the rules you must respect when writing copy:
    - Start every message by greeting the customer using their first_name. Do not include a final greeting.\n
    - When the customer's country of residence is not an English-spoken country, include a second message in their native language based on their country of residence. For example, if the country is FR, write two messages: one in English, and another in French. Never explicitly mention their country of residence.\n
    - Write short copy (SMS or push notification style) when channel is PHONE and MOBILE, and long copy (email style) for canal WEBAPP. Never explicitly mention the channel.
    - Never mention a customer's gender.\n
    - Consider the total amount of their last order when pushing for discount. For example, if the total amount is less than $100, push for discount. If the total amount is greater than $100, push for a free shipping offer.\n
    - Consider the total number of items in their last order when pushing for package deal. For example, if the total number of items is less than 5, push for package deal. If the total number of items is greater than 5, push for a free shipping offer.\n
    - Consider the days elapsed since the last purchased date when writing copy. For example, if the time elapsed is less than 30 days, push for discount. If the time elapsed is greater than 30 days, push for a free shipping offer.\n
    - Consider the current date when writing copy to take into account the seasonality of the country. For example, if the current season is Summer in the customer's country, promote for some products in summer and vice versa.\n
    - If the customer is at risk at churning, add urgency to the copy. Never explicitly mention the customer's churn status.\n
    - Make sure the input parameters match the order of the function.
    """

### Step 2: Function definition

Next, like before, we will utilize `ai_query` to define our function, leveraging the prompt from our previous cell as well as the previous two functions. 

In [0]:
spark.sql(f"""
  CREATE OR REPLACE FUNCTION generate_marketing_copy(input_user_id STRING, firstname STRING, channel STRING, country STRING, gender INT, order_amount INT, order_item_count INT, last_order_date STRING, current_date STRING, days_elapsed_since_last_order INT, churn_status STRING)
  RETURNS STRING
  LANGUAGE SQL
  COMMENT "This function generates marketing copy for a given customer User ID. This function expects the user order information which can be found with churn_predictor(), as well as their churn status which is found with customer_order_lookup(). This function returns the marketing copy as a string. "
  RETURN (
    SELECT ai_query(
      'databricks-meta-llama-3-1-405b-instruct', 
      CONCAT(
        "{prompt}"
        'Customer details: ', 
        TO_JSON(NAMED_STRUCT('firstname', firstname, 'channel', channel, 'country', country, 'gender', gender, 'order_amount', order_amount, 'order_item_count', order_item_count, 'last_order_date', last_order_date, 'current_date', current_date, 'days_elapsed_since_last_order', days_elapsed_since_last_order, 'churn_status', churn_status))
      )
    )
  )
  """)

###  Step 3: Example:
Now, let's make sure that this works. Let's input manually a customer's information that we know is likely to churn. Of course, in practice, we expect that this function will automatically utilize the output of the previous two functions, creating a _compound system_.

In [0]:
%sql
SELECT generate_marketing_copy(
  '2d17d7cd-38ae-440d-8485-34ce4f8f3b46',
  'Christopher',
  'WEBAPP',
  'USA',
  0,
  105,
  3,
  '2023-06-07',
  '2025-03-05',
  637,
  churn_predictor('2d17d7cd-38ae-440d-8485-34ce4f8f3b46')
)

## Next Steps

Proceed to notebook [05.2-Agent-Creation-Guide]($./05.2-Agent-Creation-Guide) in order to package the above functions into an implementable AI Agent with Databricks!